In [31]:
import sys
sys.path.append('../')
import pytest
import xlwings as xw
import pandas as pd, numpy as np
from pylim import lim
pd.options.display.float_format = "{:,.2f}".format
from xlwings import Range, constants
from oilanalytics.freight import tce_calc, routes, route_consts as const

In [16]:
wb = xw.Book(r'C:\dev\oilanalytics\notebooks\Baltic Exchange Tanker TCE Calculator SEP.20.xlsx')
sh = wb.sheets['TCE Calculator']

In [21]:
route = 'TC2_37'

In [20]:
df = routes.lim_data(route, start_date="date is within 4 days")
# df = df.iloc[[-2]]
df

,MGO,VLSFO,LSFO,HSFO,WorldScale,FlatRate
2021-10-08,705.25,565.25,526.75,471.00,NaN,12.31
2021-10-09,NaN,NaN,NaN,NaN,NaN,NaN
2021-10-10,NaN,NaN,NaN,NaN,NaN,NaN
2021-10-11,715.25,574.00,530.50,479.00,NaN,12.31


In [22]:
ws = df.iloc[0].WorldScale
fr = df.iloc[0].FlatRate
mgo = df.iloc[0].MGO
vlsfo = df.iloc[0].VLSFO

In [23]:
routef = route.upper()
if routef == 'TC2':
    routef = 'TC2_37'
sh.range("b7").value = routef
sh.range("C7").value = ws
sh.range("D7").value = fr

In [24]:
if route in const.route_using_sing_fuels:
    sh.range("C16").clear_contents()
    sh.range("C17").clear_contents()
    sh.range("C14").value = vlsfo
    # sh.range("C15").value = mgo
else:
    sh.range("C14").clear_contents()
    sh.range("C15").clear_contents()
    sh.range("C16").value = vlsfo
    sh.range("C17").value = mgo

In [25]:
loadportcost = sh.range("c60").value
dischargeportcost = sh.range("c61").value

In [26]:
dd = {
    'FlatRate': fr,
    'WorldScale': ws,
    'MGO': mgo-5,
    'VLSFO': vlsfo-5,    
    'HSFO': vlsfo-5,    
}
dd

{'FlatRate': 12.31,
 'WorldScale': nan,
 'MGO': 700.25,
 'VLSFO': 560.25,
 'HSFO': 560.25}

In [27]:
ds = df.index[0]
data = pd.DataFrame(dd, index=[pd.to_datetime(ds)])
res = tce_calc.calc(route, data)
res

,FlatRate,WorldScale,MGO,VLSFO,HSFO,Freight_USDMT,GrossFreight,BunkerCost,NetFreight,TotalExpenses,NetIncome,TCE
2021-10-08,12.31,NaN,700.25,560.25,560.25,NaN,NaN,"446,691.11",NaN,"541,230.17",NaN,NaN


In [28]:
cmp = sh.range("B79:C86").value
cmp = pd.DataFrame(cmp).set_index(0)
cmp = cmp.rename(columns={1:'Baltic'})
cmp['Code'] = pd.Series()
cmp.loc['Gross Freight', 'Code'] = res['GrossFreight'].iloc[0]
cmp.loc['Bunker Cost', 'Code'] = res['BunkerCost'].iloc[0]
cmp.loc['Net Income', 'Code'] = res['NetIncome'].iloc[0]
cmp.loc['Total Expenses', 'Code'] = res['TotalExpenses'].iloc[0]
cmp.loc['Baltic TCE', 'Code'] = res['TCE'].iloc[0]
cmp

C:\Users\ui906879\AppData\Local\Temp\1/ipykernel_11948/2024915855.py:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  cmp['Code'] = pd.Series()


,Baltic,Code
0,,
Gross Freight,"30,572.91",NaN
Net Freight,"29,426.43",NaN
Bunker Cost,"413,495.95","446,691.11"
Total Expenses,"508,035.02","541,230.17"
Net Income,"-478,608.59",NaN
- Gross,"-17,059.04",NaN
- Net,"-17,100.00",NaN
Baltic TCE,"-17,100.00",NaN


In [ ]:
# gross freight
assert sh.range("C79").value == pytest.approx(res['GrossFreight'].iloc[0], abs=1e-1)

In [ ]:
# bunker cost
assert sh.range("C81").value == pytest.approx(res['BunkerCost'].iloc[0], abs=1e-1)

In [ ]:
# tce
assert sh.range("C86").value == pytest.approx(res['TCE'].iloc[0], abs=1e-1)

In [ ]:
print("assert res['GrossFreight'][ds] ==  pytest.approx(%.2f, abs=1e-1)" % res['GrossFreight'].iloc[0])
print("assert res['BunkerCost'][ds] ==  pytest.approx(%.2f, abs=1e-1)" % res['BunkerCost'].iloc[0])
print("assert res['TCE'][ds] ==  pytest.approx(%.2f, abs=1e-1)" % res['TCE'].iloc[0])

In [47]:
f = r'Baltic Exchange Tanker TCE Calculator SEP.20.xlsx'
df = pd.read_excel(f, sheet_name='Raw Data', index_col='Description')
df = df[~df.index.isin(['WS Flat Rate', 'WS %', np.NaN])]
ds = pd.read_excel(f, sheet_name='Default settings', index_col='Description', skiprows=[0,1,2,3])
df = pd.merge(df.T, ds.T[dscols], left_index=True, right_index=True).T
df = df.replace(to_replace=np.NaN, value=0)
d = df.to_dict()
d


{'TD1': {'Knots Ballast': 12.5,
  'Knots Laden': 13,
  'IFO-Ballast': 53,
  'IFO-Laden': 70,
  'IFO-Port (Loading)': 20,
  'MDO-Port (Loading)': 1,
  'IFO-Port (Discharging)': 110,
  'MDO-Port (Discharging)': 1,
  'Heating Fuel Consumption': 0,
  'IFO-Anchor': 10,
  'GRT': 0,
  '$ per GRT': 0,
  'Suez Canal NT': 155000,
  'Canal Rate': 404530,
  'Canal Port Cost': 49460,
  'Voyage Details': 0,
  'Last Port': 'Loop',
  'Load Port': 'Ras Tanura',
  'Discharge Port': 'Loop',
  'Bunkers': 'Singapore',
  'Ballast Miles Non ECA': 9039,
  'Laden Miles Non ECA': 12131,
  'Ballast Miles ECA': 592,
  'Laden Miles ECA': 207,
  'Cargo Quantity (Mts)': 280000,
  'Commission %': 2.5,
  'Charges - Load port': 80521.3333333333,
  'Charges - Discharge port': 18975,
  'Canal Dues': 453990,
  'WS Fixed Differential': 0.27,
  'Additional Expenses': 0,
  'Weather Factor:': 0.05,
  'Days Ballast Non ECA': 0,
  'Days Laden Non ECA': 0,
  'Days Ballast ECA': 0,
  'Days Laden ECA': 0,
  'Days-Loading': 2,
  'D

In [35]:

ds


,TC1,TC2_37,TC4,TC5,TC6,TC7,TC8,TC9,TC10,TC11,...,TD15,TD17,TD18,TD19,TD20,TD21,TD22,TD23,TD24,TD25
Description,,,,,,,,,,,,,,,,,,,,,
Knots Ballast,12.50,12.50,12.50,12.50,12.50,12.50,12.50,12.50,12.50,12.50,...,12.50,12.50,12.50,12.50,12.50,12.50,12.50,12.50,12.50,12.50
Knots Laden,13,13,13,13,13,13,13,13,13,13,...,13,13,13,13,13,13,13,13,13,13
IFO-Ballast,36,22.50,22.50,28,21,22.50,28,21,22.50,22.50,...,53,36,21,36,44,28,53,44,36,36
IFO-Laden,43,28,28,33,25,28,33,25,28,28,...,70,43,25,43,53,33,70,53,43,43
IFO-Port (Loading),10,5,5,5,5,5,5,5,5,5,...,20,10,5,10,12,5,20,12,10,10
MDO-Port (Loading),1,2,2,2,2,2,2,2,2,2,...,1,1,2,1,1,2,1,1,1,1
IFO-Port (Discharging),42.50,25,25,32,20,25,32,20,25,25,...,110,55,20,55,68,32,110,68,55,55
MDO-Port (Discharging),1,2,2,2,2,2,2,2,2,2,...,1,1,2,1,1,2,1,1,1,1
Heating Fuel Consumption,0,0,0,0,0,0,0,0,0,0,...,0,0,34.68,0,0,55.83,0,0,0,0


In [39]:
df.head()

,TD1,TD2,TD3 C,TD15,TD22,TD6,TD20,TD23,TD7,TD8,...,TD18,TC6,TC9,TC17,TC18,TC5,TC8,TC16,TD12,TD21
Description,,,,,,,,,,,,,,,,,,,,,
Knots Ballast,12.50,12.50,12.50,12.50,12.50,12.50,12.50,12.50,12.50,12.50,...,12.50,12.50,12.50,12.50,12.50,12.50,12.50,12.50,12.50,12.50
Knots Laden,13,13,13,13,13,13,13,13,13,13,...,13,13,13,13,13,13,13,13,13,13
IFO-Ballast,53,53,53,53,53,44,44,44,36,36,...,21,21,21,22.50,22.50,28,28,28,28,28
IFO-Laden,70,70,70,70,70,53,53,53,43,43,...,25,25,25,28,28,33,33,33,33,33
IFO-Port (Loading),20,20,20,20,20,12,12,12,10,10,...,5,5,5,5,5,5,5,5,5,5


In [40]:
ds.head()

,TC1,TC2_37,TC4,TC5,TC6,TC7,TC8,TC9,TC10,TC11,...,TD15,TD17,TD18,TD19,TD20,TD21,TD22,TD23,TD24,TD25
Description,,,,,,,,,,,,,,,,,,,,,
Knots Ballast,12.50,12.50,12.50,12.50,12.50,12.50,12.50,12.50,12.50,12.50,...,12.50,12.50,12.50,12.50,12.50,12.50,12.50,12.50,12.50,12.50
Knots Laden,13,13,13,13,13,13,13,13,13,13,...,13,13,13,13,13,13,13,13,13,13
IFO-Ballast,36,22.50,22.50,28,21,22.50,28,21,22.50,22.50,...,53,36,21,36,44,28,53,44,36,36
IFO-Laden,43,28,28,33,25,28,33,25,28,28,...,70,43,25,43,53,33,70,53,43,43
IFO-Port (Loading),10,5,5,5,5,5,5,5,5,5,...,20,10,5,10,12,5,20,12,10,10


In [46]:
dscols = ['LoadingFuel','DischargeFuel','WaitingFuel','SuezWaiting','Lumpsum','OwnerPremium','HeatingFuel']


,TD1,TD2,TD3 C,TD15,TD22,TD6,TD20,TD23,TD7,TD8,...,TD18,TC6,TC9,TC17,TC18,TC5,TC8,TC16,TD12,TD21
Description,,,,,,,,,,,,,,,,,,,,,
Knots Ballast,12.50,12.50,12.50,12.50,12.50,12.50,12.50,12.50,12.50,12.50,...,12.50,12.50,12.50,12.50,12.50,12.50,12.50,12.50,12.50,12.50
Knots Laden,13,13,13,13,13,13,13,13,13,13,...,13,13,13,13,13,13,13,13,13,13
IFO-Ballast,53,53,53,53,53,44,44,44,36,36,...,21,21,21,22.50,22.50,28,28,28,28,28
IFO-Laden,70,70,70,70,70,53,53,53,43,43,...,25,25,25,28,28,33,33,33,33,33
IFO-Port (Loading),20,20,20,20,20,12,12,12,10,10,...,5,5,5,5,5,5,5,5,5,5
MDO-Port (Loading),1,1,1,1,1,1,1,1,1,1,...,2,2,2,2,2,2,2,2,2,2
IFO-Port (Discharging),110,110,110,110,110,68,68,68,55,55,...,20,20,20,25,25,32,32,32,32,32
MDO-Port (Discharging),1,1,1,1,1,1,1,1,1,1,...,2,2,2,2,2,2,2,2,2,2
Heating Fuel Consumption,0,0,0,0,0,0,0,0,0,289.88,...,6,0,0,0,0,0,0,0,8,8
